In [2]:
from cobra.io import read_sbml_model

In [3]:
model=read_sbml_model('models/yeast_lyco.xml')

# Maximum theoretical yield of lycopene using default medium

#### Calculating the maximum productivity and maximum threoretical yields in mmol/mmol:

In [4]:
with model:
    model.objective = model.reactions.CrtI #Setting the objective to the reaction, which is producing Lycopene
    lycopene_production = model.optimize().objective_value
    print('Maximum productivity of lycopene =', lycopene_production, 'mmol/gDW*h')

    glc_uptake_flux = model.reactions.r_1714.flux 
    max_yield = lycopene_production / (-1*glc_uptake_flux)
    print('Maximum theoretical yield =', max_yield, 'mmol-lyco/mmol-glc')

Maximum productivity of lycopene = 0.0871195699738864 mmol/gDW*h
Maximum theoretical yield = 0.0871195699738864 mmol-lyco/mmol-glc


#### Calculating the maximum productivity and maximum threoretical yields in C-mol/C-mol:

In [5]:
# Factors for conversion from mol to C-mol
cv_glc = 6   # Glucose
cv_lyco = 40 # Lycopene

lyco_prod_cmol = (lycopene_production)/(cv_lyco)
lyco_yield_cmol = (max_yield*cv_lyco)/(cv_glc)
print('Maximum productivity of lycopene =', lyco_prod_cmol, 'cmol-lyco/gDW*h')
print('Maximum theoretical yield =', lyco_yield_cmol, 'cmol-lyco/cmol-glc')

Maximum productivity of lycopene = 0.00217798924934716 cmol-lyco/gDW*h
Maximum theoretical yield = 0.5807971331592426 cmol-lyco/cmol-glc


The theoretical production and yield of lycopene on glucose is fairly low.\
Summarized in mmol units:

- Maximum productivity = $0.087 \frac{mmol-lyco}{g DW \cdot h}$

- Maximum yield = $0.087 \frac{mmol-lyco}{mmol-glc}$

And in cmol units:

- Maximum productivity = $0.002 \frac{cmol-lyco}{g DW \cdot h}$

- Maximum yield = $0.581 \frac{cmol-lyco}{cmol-glc}$

#### Now trying to increase the glucose concentration to increase lycopene production

Firstly, checking medium composition:

In [6]:
# Checking medium composition via the extracellular exchange reactions
for m in model.medium.items():
    print(model.reactions.get_by_id(m[0]), "|", model.reactions.get_by_id(m[0]).name, "| conc. =", m[1])

r_1654: s_0420[e] <=>  | ammonium exchange | conc. = 1000.0
r_1714: s_0565[e] <=>  | D-glucose exchange | conc. = 1.0
r_1832: s_0796[e] <=>  | H+ exchange | conc. = 1000.0
r_1861: s_0925[e] <=>  | iron(2+) exchange | conc. = 1000.0
r_1992: s_1277[e] <=>  | oxygen exchange | conc. = 1000.0
r_2005: s_1324[e] <=>  | phosphate exchange | conc. = 1000.0
r_2020: s_1374[e] <=>  | potassium exchange | conc. = 1000.0
r_2049: s_1438[e] <=>  | sodium exchange | conc. = 1000.0
r_2060: s_1468[e] <=>  | sulphate exchange | conc. = 1000.0
r_2100: s_0805[e] <=>  | water exchange | conc. = 1000.0
r_4593: s_4200[e] <=>  | chloride exchange | conc. = 1000.0
r_4594: s_4201[e] <=>  | Cu2(+) exchange | conc. = 1000.0
r_4595: s_4202[e] <=>  | Mn(2+) exchange | conc. = 1000.0
r_4596: s_4203[e] <=>  | Zn(2+) exchange | conc. = 1000.0
r_4597: s_4204[e] <=>  | Mg(2+) exchange | conc. = 1000.0
r_4600: s_4199[e] <=>  | Ca(2+) exchange | conc. = 1000.0


#### Increasing glucose concentration and investigating effects on growth rate:

Firstly, we check the medim composition: 

In [7]:
medium_glc_ex = model.reactions.get_by_id('r_1714')

In [8]:
medium = model.medium
with model: #modification are done locally within the indent
    growth_rate = model.optimize().objective_value
    print("At glucose concentration = {}:".format(medium[medium_glc_ex.id]), "Maximum theoretical growth rate =", growth_rate, "h^-1")
    
    medium[medium_glc_ex.id] = 1000 #glucose level is changed 
    model.medium = medium
    
    growth_rate = model.optimize().objective_value
    print("At glucose concentration = {}:".format(medium[medium_glc_ex.id]), "Maximum theoretical growth rate =", growth_rate, "h^-1")    

At glucose concentration = 1.0: Maximum theoretical growth rate = 0.08374775763992785 h^-1
At glucose concentration = 1000: Maximum theoretical growth rate = 19.81683025946431 h^-1


#### Increased glucose effects on lycopene production:

In [9]:
medium = model.medium
with model:
    model.objective = model.reactions.CrtI #Setting the objective to the reaction, which is producing Lycopene
    lycopene_production = model.optimize().objective_value
    print("At glucose concentration = {}:".format(medium[medium_glc_ex.id]))
    print('Maximum theoretical productivity of lycopene =', lycopene_production, 'mmol/(gDW*h^-1)')
    
    glc_uptake_flux = model.reactions.r_1714.flux 
    max_yield = lycopene_production / (-1*glc_uptake_flux)
    print('Maximum theoretical yield =', max_yield, 'mmol-lyco/mmol-glc\n')
    
    medium[medium_glc_ex.id] = 700
    model.medium = medium

    lycopene_production = model.optimize().objective_value
    glc_uptake_flux = model.reactions.r_1714.flux
    max_yield = lycopene_production / (-1*glc_uptake_flux)
    print("At glucose concentration = {}:".format(medium[medium_glc_ex.id]))
    print('Maximum theoretical productivity of lycopene =', lycopene_production, 'mmol/(gDW*h*^-1)')
    print('Maximum theoretical yield =', max_yield, 'mmol-lyco/mmol-glc\n')

At glucose concentration = 1.0:
Maximum theoretical productivity of lycopene = 0.08711956997388677 mmol/(gDW*h^-1)
Maximum theoretical yield = 0.08711956997388677 mmol-lyco/mmol-glc

At glucose concentration = 700:
Maximum theoretical productivity of lycopene = 25.82544091710761 mmol/(gDW*h*^-1)
Maximum theoretical yield = 0.04014908299958078 mmol-lyco/mmol-glc



Increasing the external glucose concentration 1000-fold can theoreticlaly increase the lycopene productivity from $0.087 \frac{mmol}{gDW \cdot h}$ to approximately $25.825 \frac{mmol}{gDW \cdot h}$.\
This is a 296-fold increase!

Although this observation does not give the complete picture. Increasing glucose concentration decreases the potential yield of lycopene from $0.087 \frac{mmol-lyco}{mmol-glc}$ to $0.040 \frac{mmol-lyco}{mmol-glc}$.

## Comparing to literature 
### Chen et al. 2016

Using the same glucose flux as found in literature to be able to compare the productivity. They had 55.56 mg lycopene/gCDW and a final titer of 1.65 g/L in 5 L bioreactors with 2% glucose concentration. It is the possible to find the productivity of lycopene they got

$$55.56\frac{mg}{gDW}\cdot \frac{48h}{536.9}\frac{mol}{g}=0.00216\frac{mmol}{h\cdot gDW}$$

We can also calculate the average glucose flux they had

$$\frac{1.65\frac{g}{L}\cdot 5 L}{0.05556\frac{g}{gCW}}=148.48gCDW$$
and the moles glucose
$$\frac{5L\cdot20\frac{g\ glucose}{L}}{180\frac{g}{mol}}=556mmol$$
so the average glucose flux
$$\frac{555mmol}{48h*148.48gDW}=0.078\frac{mmol}{h\cdot gCDW}$$

Below we introduce this flux to see what productivity our model obtains.

In [10]:
medium=model.medium

In [11]:
with model:
    solution = model.optimize()
    print('Maximum theoretical biomass productivity:',solution.fluxes["r_2111"], "\h") 
        
    medium['r_1714'] = 0.0778 #Glucose flux calculated from Chen et al. 2016
    model.medium=medium
    model.objective = model.reactions.CrtI
    lyco_production = model.optimize().objective_value
    print('Maximum theoretical productivity of lycopene:', lyco_production, 'mmol/gDW*h')
    print("Theoretical max. yield of lycopene:", lyco_production / (-1*model.reactions.r_1714.flux), "mmol-lyco / mmol-glc")

Maximum theoretical biomass productivity: 0.08374770604149266 \h
Maximum theoretical productivity of lycopene: 0.004313572261291413 mmol/gDW*h
Theoretical max. yield of lycopene: 0.05544437353845004 mmol-lyco / mmol-glc


So our model has about 2 times higher productivy at this concentration of glucose, but it is within the same order of magnitude. This shows that the researchers are getting close to the maximal lycopene flux.

### Ma et al 2019

In [13]:
with model:
    solution = model.optimize()
    print('Maximum theoretical biomass productivity:',solution.fluxes["r_2111"], "\h") 
        
    medium['r_1714'] = 0.038 #Glucose flux calculated from Chen et al. 2016
    model.medium=medium
    model.objective = model.reactions.CrtI
    lyco_production = model.optimize().objective_value
    print('Maximum theoretical productivity of lycopene:', lyco_production, 'mmol/gDW*h')
    print("Theoretical max. yield of lycopene:", lyco_production / (-1*model.reactions.r_1714.flux), "mmol-lyco / mmol-glc")

Maximum theoretical biomass productivity: 0.08374770604149098 \h
Maximum theoretical productivity of lycopene: 0.00038555796998959545 mmol/gDW*h
Theoretical max. yield of lycopene: 0.010146262368147248 mmol-lyco / mmol-glc


Calculations are similar to the one shown. They had a final titer of 2.37 g/L and 73.3 mg /gDW, fermented for 90 hours and had the same glucose concentration and same volume. They obtain a productivity of 0.0015 mmol/gDW*h which is an order of magnitude higher then what we obtain with no modifications to the model. In conclusion our model is fairly realistic when compairing to literature.